In [2]:
var inputdata = List((1,2),(1,3),(2,4))

inputdata = List((1,2), (1,3), (2,4))


List((1,2), (1,3), (2,4))

In [3]:
var kvrdd = sc.parallelize(inputdata)

kvrdd = ParallelCollectionRDD[0] at parallelize at <console>:29


ParallelCollectionRDD[0] at parallelize at <console>:29

In [4]:
kvrdd.collect()

[Stage 0:>                                                          (0 + 6) / 6]

[(1,2), (1,3), (2,4)]

In [8]:
val out = kvrdd.reduceByKey((a,b)=>a+b)

out = ShuffledRDD[2] at reduceByKey at <console>:31


ShuffledRDD[2] at reduceByKey at <console>:31

In [9]:
out.collect()

[(1,5), (2,4)]

In [10]:
hadoop fs -get /data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/FlumeData.1367523670393.gz

Name: Compile Error
Message: <console>:1: error: ';' expected but double literal found.
hadoop fs -get /data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/FlumeData.1367523670393.gz
                                                                                   ^

StackTrace: 

In [1]:
import java.io.{ByteArrayOutputStream, ByteArrayInputStream}
import java.util.zip.{GZIPOutputStream, GZIPInputStream}
import scala.collection.mutable.ArrayBuffer

In [2]:
var tweets_raw = sc.binaryFiles("/data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/")

tweets_raw = /data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/ BinaryFileRDD[3] at binaryFiles at <console>:30


/data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/ BinaryFileRDD[3] at binaryFiles at <console>:30

In [3]:
tweets_raw.take(2)(0)._1

hdfs://cxln1.c.thelab-240901.internal:8020/data/SentimentFiles/SentimentFiles/upload/data/tweets_raw/FlumeData.1367523670394.gz

In [5]:
import java.io.DataInputStream
import org.apache.spark.input.PortableDataStream

In [7]:
def count_length(compressed: PortableDataStream): Int = {
    var compressedis:DataInputStream = null
    try{
        var compressedis = compressed.open()

        val gzipInputStream = new GZIPInputStream(compressedis)
        var output = 0
        var totalByteCount = 0
        val bytes = new Array[Byte](1024)
        while (gzipInputStream.available() == 1) {
          val byteCount = gzipInputStream.read(bytes)
          if (byteCount > 0) {
            var lb = bytes.take(byteCount)
            output = output + lb.length
            totalByteCount += byteCount
          }
        }
        return output;
    }
    finally{
        if(compressedis != null)
            compressedis.close()
    }  
}

count_length: (compressed: org.apache.spark.input.PortableDataStream)Int


In [8]:
var decompressed_tweets_lengths = tweets_raw.mapValues(t => count_length(t))

decompressed_tweets_lengths = MapPartitionsRDD[4] at mapValues at <console>:36


MapPartitionsRDD[4] at mapValues at <console>:36

In [9]:
decompressed_tweets_lengths.saveAsTextFile("onlytweets_length_after_decompressing")

[Stage 4:=============================>                             (1 + 1) / 2]